Get Authorization Token

In [1]:
import sys
import spotipy
import spotipy.util as util
import requests
import json
import pandas as pd
import numpy as np

scope = 'user-library-read'
username = 'srijanduggal17'
client_id = ''
client_secret = ''
redirect_uri = 'http://localhost/'

token = util.prompt_for_user_token(username,scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

Get uris, albums, for all saved tracks.

Data Format:
In my library, for every album of each song, get every song
Label: whether i saved that song or not

List of my songs -> go through all of my spotify tracks and get the song uris
<br>List of albums to look through: -> go through all of my spotify tracks and get the album uris
<br>List of all song uris for dataset: -> go through list of albums and get all track uris
<br>Label song uris -> go through list of all song uris. Find the ones that are in list of my songs and label them with 1. Label the others as 0.
<br>Replace uris with audio features.

# Get all of my Song and Album uris

Get total number of songs in my library

In [2]:
nextOffset = 0
nextLimit = 50

results = sp.current_user_saved_tracks(limit=nextLimit, offset=nextOffset)
print('Total Tracks {}'.format(results['total']))

Total Tracks 2118


Get song and album ids for all songs in my library

In [3]:
mySongIds = set()
myAlbumIds = set()
for item in results['items']:
    mySongIds.add(item['track']['id'])
    myAlbumIds.add(item['track']['album']['id'])
nextOffset += nextLimit

while (results['next'] != None): 
    results = sp.current_user_saved_tracks(limit=nextLimit, offset=nextOffset)
    for item in results['items']:
        mySongIds.add(item['track']['id'])
        myAlbumIds.add(item['track']['album']['id'])
    nextOffset += nextLimit

In [4]:
print('Total Song Ids {}'.format(len(mySongIds)))    
# print(mySongIds)
print('Total Album Ids {}'.format(len(myAlbumIds)))

Total Song Ids 2118
Total Album Ids 842


# Get uris of all Songs from my Albums

In [24]:
albumSongIds = set()

for albumId in list(myAlbumIds):
    nextLimit = 50
    nextOffset = 0
    albumInfo = sp.album_tracks(albumId, limit=nextLimit)
    for item in albumInfo['items']:
        albumSongIds.add(item['id'])
    nextOffset += nextLimit

    while (albumInfo['next'] != None): 
        print(albumId)
        albumInfo = sp.album_tracks(albumId, limit=nextLimit, offset=nextOffset)
        for item in albumInfo['items']:
            albumSongIds.add(item['id'])
        nextOffset += nextLimit

0QfX3caqG3gvfY1Kal2tlQ
0QfX3caqG3gvfY1Kal2tlQ
22KCcelRH1AeHz7S7x5XhY
22KCcelRH1AeHz7S7x5XhY
5L1ZI7MN9XITXEvyOl1haT
6UixeNUSjrBnxeYV0ZuGHR
3D4FYzqPi2LqJWyfsY1ZdI


In [31]:
print(len(albumSongIds))
print(len(mySongIds))
unsavedSongIds = albumSongIds.difference(mySongIds)
print(len(unsavedSongIds))

8254
2118
6136


In [36]:
df_saved = pd.DataFrame()
df_saved['song_uri'] = list(mySongIds)
df_saved['label'] = 'saved'
print(df_saved.head())

df_unsaved = pd.DataFrame()
df_unsaved['song_uri'] = list(unsavedSongIds)
df_unsaved['label'] = 'unsaved'
print(df_unsaved.head())

                 song_uri  label
0  1UH4viviUjZnS9aWgPGrk0  saved
1  2IO7yf562c1zLzpanal1DT  saved
2  1kWVgfctc2JUKCqlsZGWbU  saved
3  7wJphnipgpptcsRk6Aur3w  saved
4  19ek2TMXTpXhjn7xkdNAAJ  saved
                 song_uri    label
0  0MRZ9nMoLc7qQ8Nhovv8C8  unsaved
1  6ZYbPmxFMKDbeUxZlqU0G6  unsaved
2  0XiHcuUEf1wmvmX1cOaVR9  unsaved
3  2tqPFzcsaYx479OFbNOo3P  unsaved
4  0O29lIj6AZCrjCKyZjhdCE  unsaved


In [35]:
df = pd.concat([df_saved, df_unsaved])
print(df)

                    song_uri    label
0     1UH4viviUjZnS9aWgPGrk0    saved
1     2IO7yf562c1zLzpanal1DT    saved
2     1kWVgfctc2JUKCqlsZGWbU    saved
3     7wJphnipgpptcsRk6Aur3w    saved
4     19ek2TMXTpXhjn7xkdNAAJ    saved
...                      ...      ...
6131  11Rv97V4xZDoq0kB8h551n  unsaved
6132  6TVs7Lap6ZcrpZk1xfOgtv  unsaved
6133  25vLYJnTmmraoXoynldUYU  unsaved
6134  31mqc7EKPsJf3pe6H5jEfC  unsaved
6135  7cRw4UbtGEwGIRgYimPQM7  unsaved

[8254 rows x 2 columns]
